# Analysis of results
Using the solutions found from Algorithm X we'll now collate our results.

#### Add results to neighbouring constituencies.
To speed things up later we will find what would have been the results for each pairing.

In [1]:
import numpy as np
import pandas as pd

In [15]:
const_pairs = pd.read_csv("../Data/const_pairs.csv")
const_tris = pd.read_csv("../Data/const_tris.csv")
const_quads = pd.read_csv("../Data/const_quads.csv")

bes = pd.read_csv("../Data/bes_2019.csv")
bes = bes.rename(columns = 
                             {'con_vote_19': 'Conservative',
                             'lab_vote_19': 'Labour',
                             'ld_vote_19': 'Liberal Democrat',
                             'brexit_vote_19': 'Brexit party',
                             'green_vote_19': 'Green',
                             'snp_vote_19': 'Scottish National Party',
                             'pc_vote_19': 'Plaid Cymru',
                             'dup_vote_19': 'Democratic Unionist Party',
                             'sf_vote_19': 'Sinn Fein',
                             'sdlp_vote_19': 'Social Democratic & Labour Party',
                             'uup_vote_19': 'Ulster Unionist Party',
                             'alliance_vote_19': 'Alliance',
                             'other_vote_19': 'Other'})

bes['merging_name'] = [b.lower() for b in bes['constituency_name']]
parties = ['Conservative', 'Labour', 'Liberal Democrat',
                 'Brexit party', 'Green', 'Scottish National Party', 'Plaid Cymru',
                 'Democratic Unionist Party', 'Sinn Fein', 'Social Democratic & Labour Party', 
                 'Ulster Unionist Party', 'Alliance', 'Other']


In [24]:
def add_results(df, bes, parties):
    df_long = pd.wide_to_long(df, stubnames=["name"], i=["region",'pairing'], j="test")
    df_long = df_long.reset_index()
    df_long = df_long.drop(columns='test')
    df_long['merging_name'] = [str.replace(d.lower(), 'st. ', 'st ') for d in df_long['name']]

    df_long = df_long.merge(bes[['merging_name','total_vote_19', *parties]])
    df_long = df_long.fillna(0)
    df_results = df_long.groupby('pairing')[parties].sum()
    df_results = df_results.astype(int)
    return df_results

In [80]:
def allocation(x, no_of_seats=2, method='dhondt', cutoff=None):
    total_votes = sum(x)
    # Remove all parties which fail to meet the cutoff
    if cutoff is not None and cutoff > 0:
        cutoff_votes = round(cutoff * total_votes)
        x = x[x > cutoff_votes]
        total_votes = sum(x)
    df = pd.DataFrame(x)
    df = df.rename(columns={df.columns[0]: 'Total votes'})
    
    if method == 'simple_pr':
        df = df.assign(seats = round(df['Total votes']/total_votes * no_of_seats))
        df = df[df['seats'] > 0].drop(columns='Total votes')
    else:
        df = df.assign(seats = 0, votes = df['Total votes'])
        for i in range(no_of_seats):
            name = df.index[np.argmax(df['votes'])]
            df.at[name, 'seats'] += 1
            if method == 'dhondt':
                df.at[name, 'votes'] = df.at[name, 'Total votes'] / (df.at[name, 'seats'] + 1)
            elif method == 'webster':
                df.at[name, 'votes'] = df.at[name, 'Total votes'] / (2*df.at[name, 'seats'] + 1)
        df = df[df['seats'] > 0].drop(columns=['votes','Total votes'])
    df_dict = df.to_dict()
    return df_dict['seats']

In [78]:
df_results2 = add_results(const_pairs, bes, parties)#.reset_index()
df_results3 = add_results(const_tris, bes, parties)#.reset_index()
df_results4 = add_results(const_quads, bes, parties)#.reset_index()

In [127]:
def add_method(df, method = 'dhondt', seats=2, cutoff=None, parties=None):
    if method == 'simple_pr':
        df = df.assign(simple_pr = df[parties].apply(allocation, axis=1, no_of_seats = seats, method = 'simple_pr', cutoff=cutoff))
    elif method == 'dhondt':
        df = df.assign(simple_pr = df[parties].apply(allocation, axis=1, no_of_seats = seats, method = 'dhondt'))
    elif method == 'webster':
        df = df.assign(simple_pr = df[parties].apply(allocation, axis=1, no_of_seats = seats, method = 'webster'))
    else:
        print("Incorrect method picked!!")
    df = df.reset_index()
    df2 = pd.concat([pd.DataFrame(d, index=[p]) for d, p in zip(df[method], df['pairing'])]).fillna(0)
    df2 = df2.reset_index()
    df2 = df2.rename(columns={'index': 'pairing'})
    return df2

In [143]:
df2 = add_method(df_results2, method='dhondt', parties=parties)

In [130]:
df2.head(10)

,pairing,Conservative,Labour,Liberal Democrat,Democratic Unionist Party,Alliance,Social Democratic & Labour Party,Sinn Fein,Ulster Unionist Party,Scottish National Party,Green,Plaid Cymru,Brexit party
0,1,2.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
1,2,2.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
2,3,2.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
3,4,2.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
4,5,1.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
5,6,1.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
6,7,2.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
7,8,2.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
8,9,2.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
9,10,2.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0


In [151]:
import random
selected = random.choices([i for i in range(1, max(df2['pairing']) +1)], k=300)


In [152]:
df2[df2['pairing'].isin(selected)].apply(sum)

pairing                             198641.0
Conservative                           290.0
Labour                                 190.0
Liberal Democrat                        17.0
Democratic Unionist Party               11.0
Alliance                                 5.0
Social Democratic & Labour Party         1.0
Sinn Fein                                3.0
Ulster Unionist Party                    2.0
Scottish National Party                 21.0
Green                                    1.0
Plaid Cymru                              1.0
Brexit party                             0.0
dtype: float64